# Mood Detection Chatbot

(Original: Building a Conversational Chatbot for Slack using Rasa and Python - Parul Pandey)

**Objectives** To build a chatbot called ‘Robo’ capable of checking in on people’s mood and taking the necessary actions to cheer them up.



In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

## Installations
Official Documentation: https://rasa.com/docs/core/installation/
* Python
* Rasa Starter Pack
* Spacy Language Model


### Python
* Install Anaconda 4.2.0 for Python 3.5 or Ananconda 5.2.0 for Python 3.6 (Note: on main page of Anaconda is https://www.anaconda.com/distribution/ you see Python 3.7, which is problematic, gives HTTP error while creating env. I had to delete 3.7 and get 3.6. Btw, this site does not mention which Anaconda installer has which Python version. WHY? thats the most crucial info. Anyway, I have given them here.)
* Windows: Make sure the Microsoft $VC++$ Compiler Visual Studio 2015 is installed, so python can compile any dependencies
* https://visualstudio.microsoft.com/visual-cpp-build-tools/ Download the installer and select VC++ Build tools in the list.


### Python Environment
* By installing conda, you get base or the root environment, which is the default.
* Practical tip: DO NOT install any packages in the root. ALWAYS create and env and install inside the new env.
* Env is needed esepcially for fragile packages like Python (its treated as a package) and rasa.
* So, **conda create -n rasa python=3.6 boto3** (some additional package has to be given, so "boto3")
* Env management is again a sour point. It creates complete copy (deeeep) of python 3.6 and all other packages inside "envs" folder. Goes to 1.5 GB!! Can someone optimize it?

### Rasa Starter Environment

* Clone the starter pack provided by Rasa from https://github.com/RasaHQ/starter-pack-rasa-stack
* It gives latest running application
* Let’s take a look at the folder structure and the files that were created 
* Install git, can git clone above package.
* **cd starter-pack-rasa-stack;activate rasa; pip install -r requirements.txt**

### Spacy Language Models
* Download medium english **python -m spacy download en_core_web_md**
* Link it under admin cmd **python -m spacy link en_core_web_md en --force**

### Others
* Install nb_conda_kernels to let Jupyter Notebook see the new env
* Need Rasa Core SDK for custom actions **pip install rasa_core_sdk** and run it as **python -m rasa_core_sdk.endpoint --actions actions** where actions.py has all the custom actions
* Install pygraphviz by **conda install -c alubbock pygraphviz**

# Importing the Installations

In [2]:
import rasa_nlu
import rasa_core
import spacy

# 1. Teaching the bot to understand user inputs using Rasa NLU

## Preparing the NLU Training Data

In [3]:
nlu_md = """
## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- moin
- hey there
- let's go
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- cu
- good by
- cee you later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:mood_affirm
- yes
- indeed
- of course
- that sounds good
- correct

## intent:mood_deny
- no
- never
- I don't think so
- don't like that
- no way
- not really

## intent:mood_great
- perfect
- very good
- great
- amazing
- feeling like a king
- wonderful
- I am feeling very good
- I am great
- I am amazing
- I am going to save the world
- super
- extremely good
- so so perfect
- so good
- so perfect

## intent:mood_unhappy
- my day was horrible
- I am sad
- I don't feel very well
- I am disappointed
- super sad
- I'm so sad
- sad
- very sad
- unhappy
- bad
- very bad
- awful
- terrible
- not so good
- not very good
- extremly sad
- so saad
- Quite bad - can I get a cute picture of a [bird](group:birds), please?
- Really bad and only [doggo](group:shibes) pics and change that.
- Not good. The only thing that could make me fell better is a picture of a cute [kitten](group:cats).
- so sad. Only the picture of a [puppy](group:shibes) could make it better.
- I am very sad. I need a [cat](group:cats) picture.
- Extremely sad. Only the cute [doggo](group:shibes) pics can make me feel better.
- Bad. Please show me a [bird](group:birds) pic!
- Pretty bad to be honest. Can you show me a [puppy](group:shibes) picture to make me fell better?

## intent: inform
- A [dog](group:shibes)
- [dog](group:shibes)
- [bird](group:birds)
- a [cat](group:cats)
- [cat](group:cats)
- a [bird](group:birds)
- of a [dog](group:shibes)
- of a [cat](group:cats)
- a [bird](group:birds), please
- a [dog](group:shibes), please
"""

%store nlu_md > nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


## Defining the NLU Model Configuration

In [4]:
config = """
language: "en"

pipeline:
- name: "nlp_spacy"                   # loads the spacy language model
- name: "tokenizer_spacy"             # splits the sentence into tokens
- name: "ner_crf"                   # uses the pretrained spacy NER model
- name: "intent_featurizer_spacy"     # transform the sentence into a vector representation
- name: "intent_classifier_sklearn"   # uses the vector representation to classify using SVM
- name: "ner_synonyms"                # trains the synonyms

policies:
  - name: KerasPolicy
    epochs: 100
    max_history: 5
  - name: FallbackPolicy
    fallback_action_name: 'action_default_fallback'
  - name: MemoizationPolicy
    max_history: 5
  - name: FormPolicy
""" 

%store config > config.yml

Writing 'config' (str) to file 'config.yml'.


## Training the NLU Model.

In [5]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# loading the nlu training samples
training_data = load_data("nlu.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa_nlu.training_data.loading:Training data format of nlu.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 85 (7 distinct intents)
	- Found intents: 'mood_affirm', 'goodbye', 'mood_great', 'greet', 'mood_deny', 'inform', 'mood_unhappy'
	- entity examples: 18 (1 distinct entities)
	- found entities: 'group'

INFO:rasa_nlu.utils.spacy_utils:Trying to load spacy model with name 'en'
INFO:rasa_nlu.components:Added 'nlp_spacy' to component cache. Key 'nlp_spacy-en'.
INFO:rasa_nlu.model:Starting to train component nlp_spacy
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component tokenizer_spacy
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component ner_crf
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component intent_featurizer_spacy
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train 

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.0s finished
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component ner_synonyms
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Successfully saved model into 'D:\Yogesh\ToDos\Projects\Teaching\Teaching_DataScience\Jupyter\models\nlu\default\current'


## Evaluating the NLU model on a random text

In [6]:
# A helper function for prettier output

def pprint(o):   
    print(json.dumps(o, indent=2))
    
pprint(interpreter.parse("I am very sad. Could you send me a cat picture? "))

{
  "intent": {
    "name": "mood_unhappy",
    "confidence": 0.6937719877751085
  },
  "entities": [
    {
      "start": 35,
      "end": 38,
      "value": "cats",
      "entity": "group",
      "confidence": 0.9543162015604486,
      "extractor": "ner_crf",
      "processors": [
        "ner_synonyms"
      ]
    }
  ],
  "intent_ranking": [
    {
      "name": "mood_unhappy",
      "confidence": 0.6937719877751085
    },
    {
      "name": "mood_great",
      "confidence": 0.08991000815046545
    },
    {
      "name": "goodbye",
      "confidence": 0.07990359388259029
    },
    {
      "name": "greet",
      "confidence": 0.04721143685830545
    },
    {
      "name": "inform",
      "confidence": 0.03507393308464582
    },
    {
      "name": "mood_deny",
      "confidence": 0.027333862777911954
    },
    {
      "name": "mood_affirm",
      "confidence": 0.02679517747097309
    }
  ],
  "text": "I am very sad. Could you send me a cat picture? "
}


## Evaluating the NLU model on a test data
(Here we are using the data at hand i.e nlu.md but it isr recommended to use unseen data)

In [7]:
from rasa_nlu.evaluate import run_evaluation

run_evaluation("nlu.md", model_directory)

INFO:rasa_nlu.components:Added 'nlp_spacy' to component cache. Key 'nlp_spacy-en'.
INFO:rasa_nlu.training_data.loading:Training data format of nlu.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 85 (7 distinct intents)
	- Found intents: 'mood_affirm', 'goodbye', 'mood_great', 'greet', 'mood_deny', 'inform', 'mood_unhappy'
	- entity examples: 18 (1 distinct entities)
	- found entities: 'group'

INFO:rasa_nlu.evaluate:Intent evaluation results:
INFO:rasa_nlu.evaluate:Intent Evaluation: Only considering those 85 examples that have a defined intent out of 85 examples
INFO:rasa_nlu.evaluate:F1-Score:  0.9881730469965763
INFO:rasa_nlu.evaluate:Precision: 0.9890756302521008
INFO:rasa_nlu.evaluate:Accuracy:  0.9882352941176471
INFO:rasa_nlu.evaluate:Classification report: 
              precision    recall  f1-score   support

     goodbye       1.00      0.91      0.95        11
       greet       0.93      1.00      0.96        13
      inform    

{'intent_evaluation': {'predictions': [{'text': 'hey',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.6000070861586673},
   {'text': 'hello there',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5063155877769056},
   {'text': 'hi',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.6277834043558949},
   {'text': 'hello there',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5063155877769056},
   {'text': 'good morning',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5267843319291275},
   {'text': 'good evening',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.525568372124282},
   {'text': 'moin',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5182073953744656},
   {'text': 'hey there',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5072034965565505},
   {'text': "let's go",
    'intent': 'greet',
    'predicted': 'gree

# 2. Teaching the bot to respond using Rasa Core

## 1. Writing  Stories

In [8]:
stories_md = """
## happy path               <!-- name of the story - just for debugging -->
* greet              
  - utter_greet
* mood_great               <!-- user utterance, in format intent[entities] -->
  - utter_happy
* mood_affirm
  - utter_happy
* mood_affirm
  - utter_goodbye
  
## sad path 1               <!-- this is already the start of the next story -->
* greet
  - utter_greet             <!-- action the bot should execute -->
* mood_unhappy
  - utter_ask_picture
* inform{"animal":"dog"}  
  - action_retrieve_image
  - utter_did_that_help
* mood_affirm
  - utter_happy

## sad path 2
* greet
  - utter_greet
* mood_unhappy
  - utter_ask_picture
* inform{"group":"cat"}
  - action_retrieve_image
  - utter_did_that_help
* mood_deny
  - utter_goodbye
  
## sad path 3
* greet
  - utter_greet
* mood_unhappy{"group":"puppy"}
  - action_retrieve_image
  - utter_did_that_help
* mood_affirm
  - utter_happy
  
## strange user
* mood_affirm
  - utter_happy
* mood_affirm
  - utter_unclear

## say goodbye
* goodbye
  - utter_goodbye

## fallback
- utter_unclear

"""

%store stories_md > stories.md

Writing 'stories_md' (str) to file 'stories.md'.


## 2. Defining a Domain

In [ ]:
domain_yml = """
intents:
- greet
- goodbye
- mood_affirm
- mood_deny
- mood_great
- mood_unhappy
- inform

slots:
  group:
    type: text
    
entities:
- group

actions:
- utter_greet
- utter_did_that_help
- utter_happy
- utter_goodbye
- utter_unclear
- utter_ask_picture
- action_retrieve_image
- __main__.ApiAction

templates:
  utter_greet:
  - text: "Hey! How are you?"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_unclear:
  - text: "I am not sure what you are aiming for."
  
  utter_happy:
  - text: "Great carry on!"

  utter_goodbye:
  - text: "Bye"
  
  utter_ask_picture:
  - text: "To cheer you up, I can show you a cute picture of a dog, cat or a bird. Which one do you choose?"
"""

%store domain_yml > domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


## Custom Actions

In [ ]:
actions_py="""
from rasa_core.actions import Action
from rasa_core.events import SlotSet
from IPython.core.display import Image, display

import requests

class ApiAction(Action):
    def name(self):
        return "action_retrieve_image"

    def run(self, dispatcher, tracker, domain):
        
        group = tracker.get_slot('group')
        
        r = requests.get('http://shibe.online/api/{}?count=1&urls=true&httpsUrls=true'.format(group))
        response = r.content.decode()
        response = response.replace('["',"")
        response = response.replace('"]',"")
   
        
        #display(Image(response[0], height=550, width=520))
        dispatcher.utter_message("Here is something to cheer you up: {}".format(response))
"""
%store actions_py > actions.py

Writing 'actions_py' (str) to file 'actions.py'.


Need endpoints yml to execute the actions server

In [ ]:
endpoints_yml = """
action_endpoint:
  url: "http://localhost:5055/webhook/"
"""
%store endpoints_yml > endpoints.yml

Writing 'endpoints_yml' (str) to file 'endpoints.yml'.


In [ ]:
# import sys
# python = sys.executable
# !{python} -m rasa_core_sdk.endpoint --actions actions

##  Visualising the Training Data

In [ ]:

from IPython.display import Image, display
from rasa_core.agent import Agent

agent = Agent('domain.yml')
agent.visualize("stories.md", "story_graph.png", max_history=2)
display(Image(filename="story_graph.png"))

INFO:apscheduler.scheduler:Scheduler started
Processed Story Blocks: 100%|█████| 7/7 [00:00<00:00, 448.71it/s, # trackers=1]


## Training a Dialogue Model

In [ ]:
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent

# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions
fallback = FallbackPolicy(fallback_action_name="utter_unclear",
                          core_threshold=0.2,
                          nlu_threshold=0.1)

# agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy(), fallback])

agent = Agent('domain.yml', policies = [MemoizationPolicy(max_history=2), KerasPolicy()])
# loading our neatly defined training dialogues
training_data = agent.load_data('stories.md')

agent.train(training_data)
# agent.train(
#     training_data,
#     validation_split=0.0,
#     epochs=200
# )

agent.persist('models/dialogue')

Processed Story Blocks: 100%|████| 7/7 [00:00<00:00, 448.72it/s, # trackers=12]
Processed actions: 84it [00:00, 333.86it/s, # examples=78]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 24)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                7296      
_________________________________________________________________
dense (Dense)                (None, 15)                495       
_________________________________________________________________
activation (Activation)      (None, 15)                0         
Total params: 7,791
Trainable params: 7,791
Non-trainable params: 0
_________________________________________________________________


INFO:rasa_core.policies.keras_policy:Fitting model with 240 total samples and a validation split of 0.1


Epoch 1/100
240/240 [==============================] - ETA: 35s - loss: 2.7341 - acc: 0.06 - 6s 23ms/step - loss: 2.6618 - acc: 0.1625
Epoch 2/100
240/240 [==============================] - ETA: 0s - loss: 2.6202 - acc: 0.250 - 0s 130us/step - loss: 2.5361 - acc: 0.3625
Epoch 3/100
240/240 [==============================] - ETA: 0s - loss: 2.4992 - acc: 0.312 - 0s 130us/step - loss: 2.3980 - acc: 0.4667
Epoch 4/100
240/240 [==============================] - ETA: 0s - loss: 2.3738 - acc: 0.343 - 0s 195us/step - loss: 2.2499 - acc: 0.4708
Epoch 5/100
240/240 [==============================] - ETA: 0s - loss: 2.2531 - acc: 0.343 - 0s 195us/step - loss: 2.0851 - acc: 0.4708
Epoch 6/100
240/240 [==============================] - ETA: 0s - loss: 2.1089 - acc: 0.343 - 0s 195us/step - loss: 1.9452 - acc: 0.4708
Epoch 7/100
240/240 [==============================] - ETA: 0s - loss: 2.0348 - acc: 0.343 - 0s 195us/step - loss: 1.8427 - acc: 0.4708
Epoch 8/100
240/240 [============================

Epoch 61/100
240/240 [==============================] - ETA: 0s - loss: 0.6961 - acc: 0.906 - 0s 195us/step - loss: 0.6154 - acc: 0.8833
Epoch 62/100
240/240 [==============================] - ETA: 0s - loss: 0.6637 - acc: 0.875 - 0s 130us/step - loss: 0.6047 - acc: 0.8583
Epoch 63/100
240/240 [==============================] - ETA: 0s - loss: 0.7046 - acc: 0.875 - 0s 130us/step - loss: 0.5965 - acc: 0.8833
Epoch 64/100
240/240 [==============================] - ETA: 0s - loss: 0.6787 - acc: 0.843 - 0s 195us/step - loss: 0.5784 - acc: 0.8875
Epoch 65/100
240/240 [==============================] - ETA: 0s - loss: 0.6393 - acc: 0.843 - 0s 195us/step - loss: 0.5875 - acc: 0.8625
Epoch 66/100
240/240 [==============================] - ETA: 0s - loss: 0.6547 - acc: 0.875 - 0s 130us/step - loss: 0.5558 - acc: 0.8958
Epoch 67/100
240/240 [==============================] - ETA: 0s - loss: 0.6446 - acc: 0.937 - 0s 130us/step - loss: 0.5555 - acc: 0.8792
Epoch 68/100
240/240 [===================

INFO:rasa_core.policies.keras_policy:Done fitting keras policy model
INFO:rasa_core.agent:Model directory models/dialogue exists and contains old model files. All files will be overwritten.
INFO:rasa_core.agent:Persisted model to 'D:\Yogesh\ToDos\Projects\Teaching\Teaching_DataScience\Jupyter\models\dialogue'


# Talk to your Bot

In [ ]:
#Starting the Bot

from rasa_core.agent import Agent
agent = Agent.load('models/dialogue', interpreter=model_directory)

INFO:rasa_nlu.components:Added 'nlp_spacy' to component cache. Key 'nlp_spacy-en'.


In [ ]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_text(a)
    for response in responses:
        print(response["text"])
        

Your bot is ready to talk! Type your messages here or send 'stop'
Hi
Hey! How are you?
I am sad
To cheer you up, I can show you a cute picture of a dog, cat or a bird. Which one do you choose?
dog


ERROR:rasa_core.processor:Encountered an exception while running action 'action_retrieve_image'. Bot will continue, but the actions events are lost. Make sure to fix the exception in your custom code.


Did that help you?
no
Bye
